<a href="https://colab.research.google.com/github/jacklxf/Amazon-Production-Reviews-Recommendation/blob/master/Copy_of_imbalance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.cloud import bigquery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "carbon-sensor-259109-3fcdaf905a87.json"
client = bigquery.Client()


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from joblib import dump
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.2f}'.format

from imblearn.under_sampling import OneSidedSelection
from sklearn.ensemble import AdaBoostClassifier

pca=PCA(n_components=10)
sc = StandardScaler()
sm = SMOTE(sampling_strategy='auto', k_neighbors=8)
le=LabelEncoder()

In [ ]:
first_7days="""
select MemberID,MemberCategory,Platform,SportName,Market,
StakeOdds,StakeAmtL,StakeReturnAmtL
from RAW.esports_wager_first7days
where StakeReturnAmtL<>0
"""

In [ ]:
data = client.query(first_7days).to_dataframe().set_index('MemberID')
data

,MemberCategory,Platform,SportName,Market,StakeOdds,StakeAmtL,StakeReturnAmtL
MemberID,,,,,,,
271_IM02Ololzu8888_271,Normal,MOBILE,FIFA,Prematch,2.36,50.00,68.05
237_BW2016537053_237,Normal,MOBILE,FIFA,Prematch,2.64,211.00,-211.00
2107_yibxboyqzt_2107,Normal,MOBILE,SC2,Prematch,1.12,5.00,0.58
237_F8phuoQE1AFKTB_237,Normal,INTERNET,RL,Prematch,2.78,20.25,-20.25
2107_yib40070048_2107,Normal,INTERNET,FIFA,Prematch,1.63,5.00,3.17
...,...,...,...,...,...,...,...
558_19816221763_558,Normal,MOBILE,DOTA2,Prematch,1.77,300.00,-300.00
491_28149138THB_491,Normal,INTERNET,DOTA2,Prematch,1.72,66.36,-66.36
2103_kok18973301774_2103,Normal,MOBILE,DOTA2,Prematch,3.40,300.00,-300.00


In [ ]:
data.loc[~data['SportName'].isin(['LOL','KOG','DOTA2','CSGO']),'SportName']='Others'
print('SportName has converted.')

SportName has converted.


In [ ]:
X = data.drop(['MemberCategory'],axis=1)
y = data['MemberCategory']

In [ ]:
X = pd.get_dummies(X)

In [ ]:
X.head()

,StakeOdds,StakeAmtL,StakeReturnAmtL,Platform_INTERNET,Platform_MOBILE,SportName_CSGO,SportName_DOTA2,SportName_KOG,SportName_LOL,SportName_Others,Market_LiveBall,Market_Prematch
MemberID,,,,,,,,,,,,
271_IM02Ololzu8888_271,2.36,50.00,68.05,0,1,0,0,0,0,1,0,1
237_BW2016537053_237,2.64,211.00,-211.00,0,1,0,0,0,0,1,0,1
2107_yibxboyqzt_2107,1.12,5.00,0.58,0,1,0,0,0,0,1,0,1
237_F8phuoQE1AFKTB_237,2.78,20.25,-20.25,1,0,0,0,0,0,1,0,1
2107_yib40070048_2107,1.63,5.00,3.17,1,0,0,0,0,0,1,0,1


In [ ]:
y = le.fit_transform(y)

In [ ]:
y

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
X_std = sc.fit_transform(X)
X_train,X_test,y_train,y_test=train_test_split(X_std,y,test_size=0.3,random_state=0,stratify=y)
print('Seperated X,y'+'\n')
print('X_train : {}'.format(X_train.shape))
print('X_test : {}'.format(X_test.shape))
print('y_train : {}'.format(y_train.shape))
print('y_test : {}'.format(y_test.shape))

Seperated X,y

X_train : (1860095, 12)
X_test : (797184, 12)
y_train : (1860095,)
y_test : (797184,)


In [ ]:
def BalanceCascade(X_train, y_train, X_test, num):
    negnum = y_train[y_train == 0].shape[0]
    posnum = y_train[y_train == 1].shape[0]
    neg_index = np.argwhere(y_train == 0).reshape(negnum, )
    pos_index = np.argwhere(y_train == 1).reshape(posnum, )
    pos_train = X_train[pos_index, :]
    
    FP = pow(posnum/negnum, 1/(num-1))
    classifiers = {}; thresholds = {}; test_prob = np.empty((X_test.shape[0], num))
    for i in range(num):
        classifiers[i] = AdaBoostClassifier()
        neg_train_index = np.random.permutation(neg_index)[:posnum]
        neg_train = X_train[neg_train_index, :]
        cur_X_train = np.r_[pos_train, neg_train]
        cur_y_train = np.r_[y_train[pos_index], y_train[neg_train_index]]
        classifiers[i].fit(cur_X_train, cur_y_train)
        predict_result = classifiers[i].predict_proba(X_train[neg_index, :])[:,-1]
        thresholds[i] = np.sort(predict_result)[int(neg_index.shape[0]*(1-FP))] - 0.5
        neg_index = np.argwhere(predict_result >= (thresholds[i] + 0.5)).reshape(-1, )
        test_prob[:,i] = classifiers[i].predict_proba(X_test)[:,-1] + thresholds[i]
        print("No.{} Classifier Training Finished".format(i))
    test_prob_result = np.average(test_prob, axis=1)
    return test_prob_result

In [ ]:
negnum = y_train[y_train == 0].shape[0]
posnum = y_train[y_train == 1].shape[0]

In [ ]:
negnum,posnum

(4666, 1855429)

In [ ]:
neg_index = np.argwhere(y_train == 0).reshape(negnum, )
pos_index = np.argwhere(y_train == 1).reshape(posnum, )

In [ ]:
neg_index,pos_index

(array([    408,     541,     664, ..., 1858914, 1859520, 1859999]),
 array([      0,       1,       2, ..., 1860092, 1860093, 1860094]))

In [ ]:
len(neg_index),len(pos_index)

(4666, 1855429)

In [ ]:
FP = pow(posnum/negnum, 1/(10-1))
FP

1.9446134692126367

In [ ]:
neg_train_index = np.random.permutation(neg_index)[:posnum]
neg_train_index

array([  11802,  302993, 1618389, ..., 1341545,  278255,  759727])

In [ ]:
neg_train = X_train[neg_train_index, :]
neg_train

array([[-0.13447329,  3.19797104, -4.05372169, ..., -0.07532764,
        -0.69090841,  0.69090841],
       [-0.19181622, -0.18532366,  0.07408441, ..., 13.27533965,
        -0.69090841,  0.69090841],
       [-0.37166451, -0.22739761,  0.01452506, ..., -0.07532764,
         1.44736985, -1.44736985],
       ...,
       [ 0.14702837, -0.11763947, -0.11583265, ..., -0.07532764,
        -0.69090841,  0.69090841],
       [-0.26088839,  0.03053402, -0.29181555, ..., -0.07532764,
        -0.69090841,  0.69090841],
       [-0.34103817, -0.2164218 ,  0.0337702 , ..., -0.07532764,
        -0.69090841,  0.69090841]])

In [ ]:
pos_train = X_train[pos_index, :]
pos_train

array([[-0.37818075, -0.2182511 ,  0.03121627, ..., -0.07532764,
        -0.69090841,  0.69090841],
       [-0.35341903, -0.17983575, -0.04196328, ..., -0.07532764,
         1.44736985, -1.44736985],
       [-0.21592632, -0.00879598, -0.24510404, ..., -0.07532764,
         1.44736985, -1.44736985],
       ...,
       [-0.18595161, -0.2200804 ,  0.00583454, ..., -0.07532764,
        -0.69090841,  0.69090841],
       [-0.15858339, -0.22739761,  0.02554028, ..., -0.07532764,
        -0.69090841,  0.69090841],
       [-0.56389366, -0.05350413,  0.10601303, ..., -0.07532764,
        -0.69090841,  0.69090841]])

In [ ]:
cur_X_train = np.r_[pos_train, neg_train]
cur_X_train

array([[-0.37818075, -0.2182511 ,  0.03121627, ..., -0.07532764,
        -0.69090841,  0.69090841],
       [-0.35341903, -0.17983575, -0.04196328, ..., -0.07532764,
         1.44736985, -1.44736985],
       [-0.21592632, -0.00879598, -0.24510404, ..., -0.07532764,
         1.44736985, -1.44736985],
       ...,
       [ 0.14702837, -0.11763947, -0.11583265, ..., -0.07532764,
        -0.69090841,  0.69090841],
       [-0.26088839,  0.03053402, -0.29181555, ..., -0.07532764,
        -0.69090841,  0.69090841],
       [-0.34103817, -0.2164218 ,  0.0337702 , ..., -0.07532764,
        -0.69090841,  0.69090841]])

In [ ]:
cur_y_train = np.r_[y_train[pos_index], y_train[neg_train_index]]
cur_y_train

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
clf=MLPClassifier(activation='logistic',max_iter=300)
clf.fit(cur_X_train,cur_y_train)
predict_result = clf.predict.proba(X_train[neg_index, :])[:,-1]
predict_result

AttributeError: ignored